Proyecto Parcial 2 _ MLP y Logistic Regression

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline

In [6]:
import pandas as pd


column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
    'income'  
]

df_train = pd.read_csv(
    'adult.data',            
    header=None,            
    names=column_names    
)

df_test = pd.read_csv(
    'adult.test',            
    header=None,
    names=column_names   
)




In [7]:
df_train['native-country'] = df_train['native-country'].replace(' ?', np.nan)
df_train.dropna()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [8]:
df_train=df_train.drop(columns='fnlwgt')

In [9]:

num_features = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

cat_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

target = 'income'

x = df_train[num_features + cat_features]
y = df_train[target]

print("Features (X) seleccionadas:", x.columns.tolist())
print("Target (y) seleccionado:", target)



Features (X) seleccionadas: ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
Target (y) seleccionado: income


In [12]:
numerical_transformer=StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
preprocesser=ColumnTransformer(transformers=[
        ('num', numerical_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ],)

In [13]:
def funcion_objetivo(X_hl) :
    model=MLPClassifier(random_state=10,
                    hidden_layer_sizes=X_hl,
                    max_iter=10000)
    pipeline=Pipeline(steps=[
    ('preprocesser',preprocesser),
    ('model',model)

    ])
    scores = cross_val_score(pipeline, x, y.values.ravel(), 
                               scoring='roc_auc_ovr', cv=10)
    meanscore = np.mean(scores)
    return meanscore

In [14]:
y1 = funcion_objetivo([10, 10])
y2 = funcion_objetivo([20, 5])
y3 = funcion_objetivo([30, 15])

In [15]:

X_gauss= [[10, 10],
     [20,  5],
     [30, 15]]
Y_scores = [y1, y2, y3]

kfolds = KFold(n_splits=5, shuffle=True, random_state=10)
Y_scores = np.array(Y_scores)
print("X_gauss =", X_gauss)
print("Y_scores =", Y_scores)
regresion = GaussianProcessRegressor(kernel=RBF(length_scale=1.0), random_state=10)
regresion.fit(X_gauss,Y_scores)


X_gauss = [[10, 10], [20, 5], [30, 15]]
Y_scores = [0.90811074 0.90256981 0.89696931]


GaussianProcessRegressor(kernel=RBF(length_scale=1), random_state=10)

In [16]:
# ====================================================================
# INICIO DEL CICLO DE OPTIMIZACIÓN BAYESIANA
# ====================================================================


max_ucb_value = 1.0 # Inicializamos el valor máximo en Y de los intervalos de confianza.
Y_max_real = Y_scores.max() # El mejor score real encontrado hasta ahora.
n_iteraciones = 0
max_iteraciones = 300 # Iteraciones del ciclo.

# 2. Preparar los arreglos para las nuevas observaciones
# Convertimos las listas iniciales a arreglos de numpy para facilitar la adición
X_data = np.array(X_gauss)
Y_data = Y_scores.copy() 

# --------------------------------------------------------------------
# CICLO PRINCIPAL
# --------------------------------------------------------------------

while (Y_max_real <= max_ucb_value) and (n_iteraciones < max_iteraciones):
    
    n_iteraciones += 1
    print(f"--- Iteración {n_iteraciones} ---")
    
    # A. AJUSTAR REGRESIÓN GAUSSIANA (Actualización del Modelo)
    # Se ajusta con TODOS los puntos (iniciales + nuevos)
    regresion.fit(X_data, Y_data)
    
    # Obtenemos la media y la varianza para los intervalos de confianza.
    n1_values = np.arange(10, 30, 1) 
    n2_values = np.arange(5, 15, 1)
    n1_grid, n2_grid = np.meshgrid(n1_values, n2_values)
    X_test_completo = np.c_[n1_grid.ravel(), n2_grid.ravel()]

    # FILTRADO CRÍTICO: Creamos una máscara para eliminar los puntos ya evaluados.
    # np.isin comprueba qué filas de X_test_completo YA están en X_data
    # ~ niega la máscara, seleccionando solo los que NO están en X_data
    is_not_evaluated = ~np.isin(X_test_completo, X_data, assume_unique=False).all(axis=1)
    
    # Los puntos candidatos reales a evaluar son solo los que no están en X_data
    X_test = X_test_completo[is_not_evaluated]

    # Si no quedan puntos para evaluar, terminamos el ciclo
    if len(X_test) == 0:
        print("No quedan puntos nuevos para explorar en X_test. Terminando.")
        break
    mu, sigma = regresion.predict(X_test, return_std=True)

    # C. CALCULAR UCB (intervalos de confianza)
    kappa = 1.96 
    ucb = mu + kappa * sigma
    
    # Obtener el valor UCB máximo
    max_ucb_value = np.max(ucb)
    print(f"Máximo UCB predicho en X_test: {max_ucb_value:.4f}")
    
    #Valor máximo de ucb (intervalos de confianza)
    idx_max_ucb = np.argmax(ucb)
    X_next = X_test[idx_max_ucb].reshape(1, -1)
    
    # E. EVALUAR EL PUNTO X_NEXT EN EL MUNDO REAL (Función Objetivo)
    Y_next_real = funcion_objetivo(X_next[0])
    
    print(f"  > Punto evaluado: {X_next[0]}, ROC AUC real: {Y_next_real:.4f}")
    
    # F. ACTUALIZAR LOS ARRAYS DE DATOS Y EL MÁXIMO REAL
    X_data = np.vstack([X_data, X_next]) # Añadir la X al set de entrenamiento
    Y_data = np.hstack([Y_data, Y_next_real]) # Añadir la Y al set de entrenamiento
    
    # Actualizar el máximo ROC AUC real encontrado hasta ahora
    Y_max_real = np.max(Y_data) 
    print(f"  > Máximo ROC AUC real hasta ahora: {Y_max_real:.4f}")
    
    print("-" * 50)
    
# ====================================================================
# FINAL DEL CICLO
# ====================================================================

print("\n!!! ÓPTIMIZACIÓN BAYESIANA COMPLETADA !!!")
print(f"La condición de parada se cumplió en la iteración {n_iteraciones}.")
print(f"El ROC AUC OVR máximo encontrado es: {Y_max_real:.4f}")


--- Iteración 1 ---
Máximo UCB predicho en X_test: 2.1566
  > Punto evaluado: [11  9], ROC AUC real: 0.9090
  > Máximo ROC AUC real hasta ahora: 0.9090
--------------------------------------------------
--- Iteración 2 ---
Máximo UCB predicho en X_test: 0.9215
  > Punto evaluado: [18 14], ROC AUC real: 0.9046
  > Máximo ROC AUC real hasta ahora: 0.9090
--------------------------------------------------
--- Iteración 3 ---
Máximo UCB predicho en X_test: 0.9138
  > Punto evaluado: [10  6], ROC AUC real: 0.9077
  > Máximo ROC AUC real hasta ahora: 0.9090
--------------------------------------------------
--- Iteración 4 ---
Máximo UCB predicho en X_test: 0.9096
  > Punto evaluado: [16  9], ROC AUC real: 0.9036
  > Máximo ROC AUC real hasta ahora: 0.9090
--------------------------------------------------
--- Iteración 5 ---
Máximo UCB predicho en X_test: 0.9725
  > Punto evaluado: [29  5], ROC AUC real: 0.9001
  > Máximo ROC AUC real hasta ahora: 0.9090
------------------------------------

-----------------------------------------------------------------------------------------------------------------------------------------------------------

Modelo Logistic Reg

In [20]:
from sklearn.linear_model import LogisticRegression

In [25]:
def funcion_objetivo(X_hl) :
    model= LogisticRegression(class_weight='balanced', max_iter=1000,C=X_hl)
    pipeline=Pipeline(steps=[
    ('preprocesser',preprocesser),
    ('model',model)

    ])
    scores = cross_val_score(pipeline, x, y.values.ravel(), 
                               scoring='roc_auc_ovr', cv=10)
    meanscore = np.mean(scores)
    return meanscore

In [26]:
y1 = funcion_objetivo(0.001)
y2 = funcion_objetivo(1)
y3 = funcion_objetivo(10)
X_gauss= np.array([[.001],[1],[10]])
Y_scores = [y1, y2, y3]

kfolds = KFold(n_splits=10, shuffle=True, random_state=10)
Y_scores = np.array(Y_scores)
print("X_gauss =", X_gauss)
print("Y_scores =", Y_scores)
regresion = GaussianProcessRegressor(kernel=RBF(length_scale=1.0), random_state=10)

X_gauss = [[1.e-03]
 [1.e+00]
 [1.e+01]]
Y_scores = [0.89359266 0.9066904  0.90644119]


In [27]:
# ====================================================================
# INICIO DEL CICLO DE OPTIMIZACIÓN BAYESIANA
# ====================================================================


max_ucb_value = 1.0 # Inicializamos el valor máximo en Y de los intervalos de confianza.
Y_max_real = Y_scores.max() # El mejor score real encontrado hasta ahora.
n_iteraciones = 0
max_iteraciones = 300 # Iteraciones del ciclo.

# 2. Preparar los arreglos para las nuevas observaciones
# Convertimos las listas iniciales a arreglos de numpy para facilitar la adición
X_data = np.array(X_gauss)
Y_data = Y_scores.copy() 

# --------------------------------------------------------------------
# CICLO PRINCIPAL
# --------------------------------------------------------------------

while (Y_max_real <= max_ucb_value) and (n_iteraciones < max_iteraciones):
    
    n_iteraciones += 1
    print(f"--- Iteración {n_iteraciones} ---")
    
    # A. AJUSTAR REGRESIÓN GAUSSIANA (Actualización del Modelo)
    # Se ajusta con TODOS los puntos (iniciales + nuevos)
    regresion.fit(X_data, Y_data)
    
    # Obtenemos la media y la varianza para los intervalos de confianza.
    n1_values = np.arange(0.001, 100, 0.01) 
    #n2_values = np.arange(5, 15, 1)
    #n1_grid, n2_grid = np.meshgrid(n1_values, n2_values)
    X_test_completo = n1_values.reshape([-1, 1])#np.c_[n1_grid.ravel(), n2_grid.ravel()]

    # FILTRADO CRÍTICO: Creamos una máscara para eliminar los puntos ya evaluados.
    # np.isin comprueba qué filas de X_test_completo YA están en X_data
    # ~ niega la máscara, seleccionando solo los que NO están en X_data
    is_not_evaluated = ~np.isin(X_test_completo, X_data, assume_unique=False).all(axis=1)
    
    # Los puntos candidatos reales a evaluar son solo los que no están en X_data
    X_test = X_test_completo[is_not_evaluated]

    # Si no quedan puntos para evaluar, terminamos el ciclo
    if len(X_test) == 0:
        print("No quedan puntos nuevos para explorar en X_test. Terminando.")
        break
    mu, sigma = regresion.predict(X_test, return_std=True)

    # C. CALCULAR UCB (intervalos de confianza)
    kappa = 1.96 
    ucb = mu + kappa * sigma
    
    # Obtener el valor UCB máximo
    max_ucb_value = np.max(ucb)
    print(f"Máximo UCB predicho en X_test: {max_ucb_value:.4f}")
    
    #Valor máximo de ucb (intervalos de confianza)
    idx_max_ucb = np.argmax(ucb)
    X_next = X_test[idx_max_ucb].reshape(1, -1)
    
    # E. EVALUAR EL PUNTO X_NEXT EN EL MUNDO REAL (Función Objetivo)
    Y_next_real = funcion_objetivo(X_next[0][0])
    
    print(f"  > Punto evaluado: {X_next[0]}, ROC AUC real: {Y_next_real:.4f}")
    
    # F. ACTUALIZAR LOS ARRAYS DE DATOS Y EL MÁXIMO REAL
    X_data = np.vstack([X_data, X_next]) # Añadir la X al set de entrenamiento
    Y_data = np.hstack([Y_data, Y_next_real]) # Añadir la Y al set de entrenamiento
    
    # Actualizar el máximo ROC AUC real encontrado hasta ahora
    Y_max_real = np.max(Y_data) 
    print(f"  > Máximo ROC AUC real hasta ahora: {Y_max_real:.4f}")
    
    print("-" * 50)
    
# ====================================================================
# FINAL DEL CICLO
# ====================================================================

print("\n!!! ÓPTIMIZACIÓN BAYESIANA COMPLETADA !!!")
print(f"La condición de parada se cumplió en la iteración {n_iteraciones}.")
print(f"El ROC AUC OVR máximo encontrado es: {Y_max_real:.4f}")


--- Iteración 1 ---
Máximo UCB predicho en X_test: 1.9345
  > Punto evaluado: [99.991], ROC AUC real: 0.9064
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 2 ---
Máximo UCB predicho en X_test: 1.8860
  > Punto evaluado: [73.011], ROC AUC real: 0.9064
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 3 ---
Máximo UCB predicho en X_test: 1.2961
  > Punto evaluado: [86.651], ROC AUC real: 0.9064
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 4 ---
Máximo UCB predicho en X_test: 0.9998
  > Punto evaluado: [55.171], ROC AUC real: 0.9064
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 5 ---
Máximo UCB predicho en X_test: 0.9461
  > Punto evaluado: [62.591], ROC AUC real: 0.9064
  > Máximo ROC AUC real hasta ahora: 0.9067
-------------------------------

c:\Users\mateo\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


  > Punto evaluado: [4.071], ROC AUC real: 0.9065
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 42 ---
Máximo UCB predicho en X_test: 0.9220
  > Punto evaluado: [74.751], ROC AUC real: 0.9064
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 43 ---
Máximo UCB predicho en X_test: 0.9214
  > Punto evaluado: [22.091], ROC AUC real: 0.9065
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 44 ---
Máximo UCB predicho en X_test: 0.9180
  > Punto evaluado: [60.841], ROC AUC real: 0.9064
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 45 ---
Máximo UCB predicho en X_test: 0.9151
  > Punto evaluado: [11.881], ROC AUC real: 0.9065
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 46 ---
Máximo UCB pre

c:\Users\mateo\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


  > Punto evaluado: [27.451], ROC AUC real: 0.9065
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 58 ---
Máximo UCB predicho en X_test: 0.9070
  > Punto evaluado: [99.661], ROC AUC real: 0.9064
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 59 ---
Máximo UCB predicho en X_test: 0.9069
  > Punto evaluado: [63.941], ROC AUC real: 0.9064
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 60 ---
Máximo UCB predicho en X_test: 0.9068


...

  > Punto evaluado: [1.181], ROC AUC real: 0.9066
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 75 ---
Máximo UCB predicho en X_test: 0.9068
  > Punto evaluado: [35.161], ROC AUC real: 0.9064
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 76 ---
Máximo UCB predicho en X_test: 0.9068


c:\Users\mateo\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


  > Punto evaluado: [12.901], ROC AUC real: 0.9064
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 77 ---
Máximo UCB predicho en X_test: 0.9067
  > Punto evaluado: [93.361], ROC AUC real: 0.9064
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 78 ---
Máximo UCB predicho en X_test: 0.9067
  > Punto evaluado: [77.281], ROC AUC real: 0.9065
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 79 ---
Máximo UCB predicho en X_test: 0.9067


c:\Users\mateo\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


  > Punto evaluado: [1.071], ROC AUC real: 0.9067
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------
--- Iteración 80 ---
Máximo UCB predicho en X_test: 0.9067
  > Punto evaluado: [1.101], ROC AUC real: 0.9066
  > Máximo ROC AUC real hasta ahora: 0.9067
--------------------------------------------------

!!! ÓPTIMIZACIÓN BAYESIANA COMPLETADA !!!
La condición de parada se cumplió en la iteración 80.
El ROC AUC OVR máximo encontrado es: 0.9067
